# Part 2: Making Long-Running MCP Tools Durable with Human-in-the-Loop

In this chapter, we'll build an invoice processing system that demonstrates how Temporal makes MCP tools production-ready for these real-world scenarios.

In this workshop, we'll do the following:

1. Add a **durable timer** to enable long-running operations without resource waste
2. Send **Signals and Queries** to enable human-in-the-loop patterns while maintaining durability
3. Build **long-running MCP tools** that survive crashes, restarts, and infrastructure failures

## Hands-on Moments

This is a hands-on workshop!

All of the instructors slides and code samples are are executable in the workshop notebooks.
We encourage you to follow along and play with the samples!

At the end of every chapter (notebook) will be a hands-on lab.

In [16]:
# This allows us to run the Temporal Asyncio event loop within the event loop of Jupyter Notebooks
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Mermaid renderer, run at the beginning to setup rendering of diagrams
import base64
from IPython.display import Image, display

def render_mermaid(graph_definition):
    """
    Renders a Mermaid diagram in Google Colab using mermaid.ink.

    Args:
        graph_definition (str): The Mermaid diagram code (e.g., "graph LR; A-->B;").
    """
    graph_bytes = graph_definition.encode("ascii")
    base64_bytes = base64.b64encode(graph_bytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))

## The Challenge: Long-Running Operations in MCP Tools

In Chapter 1, we saw how Temporal makes MCP tools durable for simple request-response patterns like weather fetching. But real business processes are rarely that simple. They involve:

- **Long-running operations** that take hours or days
- **Human approvals and decisions** at various stages
- **State management** across system restarts

Workflows are complex business processes that can be long-running. Consider this real scenario:

- You build an MCP tool that processes invoices
- Processing takes 2-3 hours (validation, fraud checks, approvals)
- Your MCP server crashes 1 hour into processing
- What happens to the invoice? Where is the state stored?

## Long-Running Processes as Tool

By implementing an MCP tool as a Workflow, it can manage multi-step, long-running processes. For example, an AI might start an invoice approval Workflow that waits several days, or more, for human approval. The Workflow persists and retains its state throughout the waiting period.

### Traditional MCP Tool Problems with Long-Running Operations

Let's review what happens when MCP tools handle long-running operations without proper durability:

```python
# Traditional MCP tool approach
class TraditionalInvoiceProcessor:
    def __init__(self):
        self.processing_invoices = {}  # In-memory state - lost on restart!
    
    async def process_invoice(self, invoice_id: str):
        self.processing_invoices[invoice_id] = "validating"
        
        await asyncio.sleep(3600)  # Simulate 1-hour validation
        
        self.processing_invoices[invoice_id] = "approved"
        
        # What if the server crashes here? Do we re-process everything?
        await asyncio.sleep(7200)  # Simulate 2-hour payment processing
        
        return "complete"
```

### The Problems Stack Up:

1. **State Loss**: Server restart = all progress lost
2. **No Resume Logic**: Can't pick up where we left off
3. **Resource Waste**: Keeping connections open for hours
4. **Visibility Gap**: No way to query progress during execution

### Invoice Processing at Scale

Let's make this concrete with a real example. Imagine you're building an MCP tool for a company that processes thousands of invoices daily:

**Business Requirements:**
- **Wait 5 days for someone to approve the invoice** A durable timer needed, one that can survive server crashes
- **Query status anytime** without disrupting the process
- **Retry failed calls** to the payment microservices (due to network issues)
- **Auto-approve** if no approval happens after 5 days

**The problems with traditional approach**:
- The 5-day timer can be lost on server restart, meaning the approval deadline can be missed
- Failed microservice calls starts the entire process over
- State inconsistency

## Demo (Expand for instructor notes or to run on your own).
<!--
Prep:
1. Clone this repository: `https://github.com/Aslan11/temporal-invoice-mcp/`.
2. Ahead of time, edit `claude_desktop_config.json` and add the full path to the directory containing `server.py`.
3. Make sure you have Claude Desktop installed to your desktop as we will use this as our MCP Client.
4. Run `cp claude_desktop_config.json ~/Library/Application\ Support/Claude/` to connect the tools in this repository to the MCP client in Claude Desktop. Restart Claude Desktop if you have not already.

Demo: 
1. Run your temporal server: `temporal server start-dev`. 
2. In Claude Desktop, prompt it to `Process this invoice for me: {
  "invoice_id": "INV-100",
  "customer": "ACME Corp",
  "lines": [
    {"description": "Widget A", "amount": 100, "due_date": "2024-06-30T00:00:00Z"},
    {"description": "Widget B", "amount": 200, "due_date": "2024-07-05T00:00:00Z"}
  ]
}`.
3. Allow Claude to use the `process_invoice` tool.
4. Go on the Web UI and show that the timer has started to represent a long-running Workflow Execution.
5. Quit your Worker with `Ctrl-C`. Refresh your Web UI and show that the Workflow is still running.
6. Restart your Worker with `python worker.py` and show that the process did not restart. The timer does not restart from the beginning.
-->

## Let's create a durable long-running Tool with Temporal Workflows!

In [18]:
## Run this code to load it into the program
from temporalio import activity

@activity.defn
async def payment_gateway(line: dict) -> bool:
    activity.logger.info("Paying %s", line.get("description"))
    # Simulate payment processing
    activity.logger.info("Payment succeeded")
    return True

## Durable Timers

Durable timers survive crashes and restarts! Also, during these waiting periods, the Workflow instance is not consuming CPU or memory. The Worker only “wakes up” when a Workflow needs to perform something.

Traditional timers:
```
async def wait_for_approval():
    await asyncio.sleep(432000)  # 5 days
    # If process crashes, timer is LOST
```

Durable timers:

`await workflow.sleep(timedelta(days=5))`

In [ ]:
## Step 1: Call the durable timer for two seconds
## Step 2: Execute the `payment_gateway` Activity
## Step 3: In your retry policy, your maximum interval is the the maximum interval between retries. 
## Set your maximum interval to 30 seconds.
## Step 3: Now run the code to load it into the program

from datetime import timedelta
from temporalio import workflow
from temporalio.common import RetryPolicy
from temporalio import workflow

# sandboxed=False is a Notebook only requirement. You normally don't do this
@workflow.defn(sandboxed=False)
class InvoiceWorkflow:
    @workflow.run
    async def run(self, invoice: dict) -> str:
        # Process each line item
        for line in invoice.get("lines", []):
            # TODO: Call the durable timer for 2 seconds
            try:
                await workflow.execute_activity(
                    # TODO: Call your payment_gateway Activity
                    line,
                    start_to_close_timeout=timedelta(seconds=30),
                    retry_policy=RetryPolicy(
                        initial_interval=timedelta(seconds=1),
                        # TODO: Set your maximum interval to 30 seconds
                        maximum_attempts=3,
                    ),
                )
            except Exception as e:
                workflow.logger.error(f"Failed to process line item {line.get('item_id')}: {e}")
        return "COMPLETED"

In [19]:
# Run this cell to load and execute the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "notebooks" / "02_MCP_Temporal_HITL_Solution" / "solution1.py"

if not solution_file.exists():
    raise FileNotFoundError(f"Solution file not found at {solution_file}")

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

exec(code)
print("Solution executed successfully")


In [20]:
## Creating your MCP Tool

# Note: MCP servers cannot be run directly in Jupyter notebooks because
# MCP servers need to run as separate processes that communicate with stdio protocol
# Therefore, later on in the workshop, you will add this yourself to the `mcp_servers` directory.

from temporalio.client import Client
from fastmcp import FastMCP
from typing import Dict
import uuid

# Initialize FastMCP server
mcp = FastMCP("invoice")

# Temporal client setup (do this once, then reuse)
temporal_client = None

async def get_temporal_client():
    """Get or create a Temporal client connection."""
    global temporal_client
    if not temporal_client:
        temporal_client = await Client.connect("localhost:7233")
    return temporal_client

@mcp.tool()
async def process_invoice(invoice: Dict) -> Dict[str, str]:
    """Start the InvoiceWorkflow with the given invoice JSON."""
    client = await get_temporal_client()
    handle = await client.start_workflow(
        "InvoiceWorkflow",
        invoice,
        id=f"invoice-{uuid.uuid4()}",
        task_queue="invoice-task-queue",
    )
    return {"workflow_id": handle.id, "run_id": handle.result_run_id}

## The Human Factor: Why HITL is Critical

A Workflow can wait days and months for a Signal or event and even if the system crashes during human approval wait time, the Workflow resumes exactly where it left off. 

**Traditional Problem**:

1. CFO approves invoice → payment gateway fails → approval lost
2. Do we re-ask for approval (annoying) or assume approval (risky)?

**Temporal Solution**:

1. CFO clicks "Approve" → Signal sent to Temporal Workflow first
2. Approval durably stored → Only then does portal show success
3. Payment processing begins → Happens after approval is safe
4. Gateway fails? → Temporal retries automatically using stored approval

We do this with Signals and Queries.

In [ ]:
## Human-in-the-Loop: Signal and Query Review

### Signals - Changing Workflow State
Signals **modify** the Workflow's execution path and are recorded in history:
- Durable - persisted in Temporal, survives crashes
- Triggers action - can wake up waiting Workflows
- Examples: approve, reject, cancel, update

### Queries - Reading Workflow State
Queries **read** the current state without affecting execution:
- Read-only: never changes the Workflow
- Not recorded in history
- Examples: getStatus, getDetails, getProgress

## Demo (Expand for instructor notes or to run on your own).
<!--
Prep:
1. Clone this repository: `https://github.com/Aslan11/temporal-invoice-mcp/`.
2. Ahead of time, edit `claude_desktop_config.json` and add the full path to the directory containing `server.py`.
3. Make sure you have Claude Desktop installed to your desktop as we will use this as our MCP Client.
4. Run `cp claude_desktop_config.json ~/Library/Application\ Support/Claude/` to connect the tools in this repository to the MCP client in Claude Desktop. Restart Claude Desktop if you have not already.

Demo: 
1. Run your temporal server: `temporal server start-dev`. 
2. In Claude Desktop, prompt it to `Process this invoice for me: {
  "invoice_id": "INV-100",
  "customer": "ACME Corp",
  "lines": [
    {"description": "Widget A", "amount": 100, "due_date": "2024-06-30T00:00:00Z"},
    {"description": "Widget B", "amount": 200, "due_date": "2024-07-05T00:00:00Z"}
  ]
}`.
3. Allow Claude to use the `process_invoice` tool.
4. Go on the Web UI and show that the timer has started to represent a long-running Workflow Execution.
5. Quit Claude Desktop once again to show that the Timer is durably stored. The Workflow lives in Temporal not the MCP server.
6. Go back on Claude Desktop and ask it, "What is the status of this invoice?"
7. Allow prompt to use the `invoice_status` tool and point out the status.
8. Finally, prompt Claude to "approve this invoice."
9. Allow Claude to use the `approve_invoice` tool.
10. Show that the Workflow Execution has completed successfully.

In [ ]:
## Let's add Signals to approve or reject invoices!
## Step 1: Add a RejectInvoice Signal which updates the Workflow’s approved field to False.
## Step 2: Run this code to load the code into the program
from datetime import timedelta
from temporalio import workflow
from temporalio.common import RetryPolicy

# sandboxed=False is a Notebook only requirement. You normally don't do this
@workflow.defn(sandboxed=False)
class InvoiceWorkflow:
    def __init__(self) -> None:
        self.approved: bool | None = None

    @workflow.signal
    async def ApproveInvoice(self) -> None:
        self.approved = True

    # TODO: Add a RejectInvoice Signal which updates the Workflow’s approved field to False.

    @workflow.run  # ← Fixed indentation (4 spaces)
    async def run(self, invoice: dict) -> str:
        # Wait for the approval signal
        await workflow.wait_condition(
            lambda: self.approved is not None,
            timeout=timedelta(days=5),
        )

        # Process each line item
        for line in invoice.get("lines", []):
            await workflow.sleep(timedelta(seconds=2))

            # Try to process payment
            try:
                await workflow.execute_activity(
                    payment_gateway,
                    line,
                    start_to_close_timeout=timedelta(seconds=30),
                    retry_policy=RetryPolicy(
                        initial_interval=timedelta(seconds=1),
                        maximum_interval=timedelta(seconds=30),
                        maximum_attempts=3,
                    ),
                )
            except Exception:
                workflow.logger.error(f"Failed to process line item: {line}")

        return "COMPLETED"

In [26]:
from datetime import timedelta
from temporalio import workflow
from temporalio.common import RetryPolicy

# sandboxed=False is a Notebook only requirement. You normally don't do this
@workflow.defn(sandboxed=False)
class InvoiceWorkflow:
    def __init__(self) -> None:
        self.approved: bool | None = None

    @workflow.signal
    async def ApproveInvoice(self) -> None:
        self.approved = True

    @workflow.signal
    async def RejectInvoice(self) -> None:
        self.approved = False

    @workflow.run  # ← Fixed indentation (4 spaces)
    async def run(self, invoice: dict) -> str:
        # Wait for the approval signal
        await workflow.wait_condition(
            lambda: self.approved is not None,
            timeout=timedelta(days=5),
        )

        # Process each line item
        for line in invoice.get("lines", []):
            await workflow.sleep(timedelta(seconds=2))

            # Try to process payment
            try:
                await workflow.execute_activity(
                    payment_gateway,
                    line,
                    start_to_close_timeout=timedelta(seconds=30),
                    retry_policy=RetryPolicy(
                        initial_interval=timedelta(seconds=1),
                        maximum_interval=timedelta(seconds=30),
                        maximum_attempts=3,
                    ),
                )
            except Exception:
                workflow.logger.error(f"Failed to process line item: {line}")

        return "COMPLETED"

In [27]:
## Creating your MCP Tool to approve/reject invoices
## Step 1: Add an MCP tool which sends RejectSignal to the Workflow

# Note: MCP servers cannot be run directly in Jupyter notebooks because
# MCP servers need to run as separate processes that communicate with stdio protocol
# Therefore, later on in the workshop, you will add this yourself to the `mcp_servers` directory.

from temporalio.client import Client
from fastmcp import FastMCP
from typing import Dict

# Initialize FastMCP server
mcp = FastMCP("invoice")

# Temporal client setup (do this once, then reuse)
temporal_client = None

async def get_temporal_client():
    """Get or create a Temporal client connection."""
    global temporal_client
    if not temporal_client:
        temporal_client = await Client.connect("localhost:7233")
    return temporal_client

@mcp.tool()
async def approve_invoice(workflow_id: str, run_id: str) -> str:
    """Signal approval for the invoice workflow."""
    client = await get_temporal_client()
    handle = client.get_workflow_handle(workflow_id=workflow_id, run_id=run_id)
    await handle.signal("ApproveInvoice")
    return "APPROVED"

# TODO: Add an MCP tool to send the RejectInvoice Signal

In [2]:
# Run this cell to load and execute the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "notebooks" / "02_MCP_Temporal_HITL_Solution" / "solution2.py"

if not solution_file.exists():
    raise FileNotFoundError(f"Solution file not found at {solution_file}")

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

exec(code)
print("Solution executed successfully")


Solution loaded:


```python
from temporalio.client import Client
from fastmcp import FastMCP
from typing import Dict

# Initialize FastMCP server
mcp = FastMCP("invoice")

# Temporal client setup (do this once, then reuse)
temporal_client = None

async def get_temporal_client():
    """Get or create a Temporal client connection."""
    global temporal_client
    if not temporal_client:
        temporal_client = await Client.connect("localhost:7233")
    return temporal_client

@mcp.tool()
async def approve_invoice(workflow_id: str, run_id: str) -> str:
    """Signal approval for the invoice workflow."""
    client = await get_temporal_client()
    handle = client.get_workflow_handle(workflow_id=workflow_id, run_id=run_id)
    await handle.signal("ApproveInvoice")
    return "APPROVED"

@mcp.tool()
async def reject_invoice(workflow_id: str, run_id: str) -> str:
    """Signal rejection for the invoice workflow."""
    client = await get_temporal_client()
    handle = client.get_workflow_handle(workflow_id=workflow_id, run_id=run_id)
    await handle.signal("RejectInvoice")
    return "REJECTED"

```

Solution executed successfully


In [ ]:
## Adding Queries
## Step 1: Add a `GetInvoiceStatus` Query that returns the value of `self.status`.
## Step 2: Run this code to load the code into the program

from datetime import timedelta
from temporalio import workflow
from temporalio.common import RetryPolicy

# sandboxed=False is a Notebook only requirement. You normally don't do this
@workflow.defn(sandboxed=False)
class InvoiceWorkflow:
    def __init__(self) -> None:
        self.approved: bool | None = None
        self.status: str = "INITIALIZING"

    @workflow.signal
    async def ApproveInvoice(self) -> None:
        self.approved = True

    @workflow.signal
    async def RejectInvoice(self) -> None:
        self.approved = False

    @workflow.query
    # TODO Add a `GetInvoiceStatus` Query that returns the value of `self.status`.

    @workflow.run
    async def run(self, invoice: dict) -> str:
        self.status = "PENDING-APPROVAL"

        # Wait for the approval signal
        await workflow.wait_condition(
            lambda: self.approved is not None,
            timeout=timedelta(days=5),
        )

        if not self.approved:
            workflow.logger.info("REJECTED")
            self.status = "REJECTED"
            return "REJECTED"

        self.status = "APPROVED"
        # Process each line item
        for line in invoice.get("lines", []):
            # Simple delay - wait 2 seconds before processing
            await workflow.sleep(timedelta(seconds=2))

            # Try to process payment
            try:
                await workflow.execute_activity(
                    payment_gateway,
                    line,
                    start_to_close_timeout=timedelta(seconds=30),
                    retry_policy=RetryPolicy(
                        initial_interval=timedelta(seconds=1),
                        maximum_interval=timedelta(seconds=30),
                        maximum_attempts=3,
                    ),
                )
            except Exception:
                workflow.logger.error(f"Failed to process line item: {line}")

        self.status = "PAID"
        return self.status

In [29]:
# Run this cell to load and execute the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "notebooks" / "02_MCP_Temporal_HITL_Solution" / "solution3.py"

if not solution_file.exists():
    raise FileNotFoundError(f"Solution file not found at {solution_file}")

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

exec(code)
print("Solution executed successfully")


In [30]:
## Create your MCP Tool to query for status

# Note: MCP servers cannot be run directly in Jupyter notebooks because
# MCP servers need to run as separate processes that communicate with stdio protocol
# Therefore, later on in the workshop, you will add this yourself to the `mcp_servers` directory.

from temporalio.client import Client
from fastmcp import FastMCP
from typing import Dict

# Initialize FastMCP server
mcp = FastMCP("invoice")

# Temporal client setup (do this once, then reuse)
temporal_client = None

async def get_temporal_client():
    """Get or create a Temporal client connection."""
    global temporal_client
    if not temporal_client:
        temporal_client = await Client.connect("localhost:7233")
    return temporal_client


@mcp.tool()
async def invoice_status(workflow_id: str, run_id: str) -> str:
    """Return current status of the workflow."""
    client = await get_temporal_client()
    handle = client.get_workflow_handle(workflow_id=workflow_id, run_id=run_id)
    desc = await handle.describe()
    status = await handle.query("GetInvoiceStatus")
    return (
        f"Invoice with ID {workflow_id} is currently {status}. "
        f"Workflow status: {desc.status.name}"
    )

In [31]:
## Running your Worker
## Step 1: Set the task queue to be "invoice-task-queue"
## Step 2: Register your `InvoiceWorkflow`
## Step 4: Register your `pament_gateway` Activity
## Step 5: Run this code to load it into your program

import asyncio
from temporalio.client import Client
from temporalio.worker import Worker

async def run_worker():
    # Connect to Temporal server (change address if using Temporal Cloud)
    client = await Client.connect("localhost:7233")

    worker = Worker(
        client,
        task_queue=#TODO: Set the task queue to be "invoice-task-queue",
        workflows=[], #TODO: Register your `InvoiceWorkflow`
        activities=[], #TODO: Register your `pament_gateway` Activity
    )
    print("Worker started. Listening for workflows...")
    await worker.run()

In [1]:
# Run this cell to load and execute the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "notebooks" / "02_MCP_Temporal_HITL_Solution" / "solution4.py"

if not solution_file.exists():
    raise FileNotFoundError(f"Solution file not found at {solution_file}")

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

exec(code)
print("Solution executed successfully")


Solution loaded:


```python
from temporalio.client import Client
from temporalio.worker import Worker

async def run_worker():
    # Connect to Temporal server (change address if using Temporal Cloud)
    client = await Client.connect("localhost:7233")

    worker = Worker(
        client,
        task_queue="invoice-task-queue",
        workflows=[InvoiceWorkflow],
        activities=[payment_gateway],
    )
    print("Worker started. Listening for workflows...")
    await worker.run()

```

Solution executed successfully


In [32]:
# Due to the limitation of Jupyter Notebooks and Google Collab, this is how
# you must start the worker in a Notebook environment
worker = asyncio.create_task(run_worker())

# If you are running this code in a typical Python environment, you can start
# the Worker by just calling `asyncio.run`
# if __name__ == "__main__":
#    asyncio.run(run_worker())

Worker started. Listening for workflows...


In [ ]:
## Configure Claude Desktop 
### Take 10 minutes to do the following steps.

1. Add your new MCP tools -  (`process_invoice`, `approve_invoice`, `reject_invoice`, `invoice_status`) from the section above to a new file - called `invoice.py` - in the `mcp_servers` directory. Follow the structure used in `weather.py`.
2. At the bottom of `invoice.py`, make sure you initialize and run the server like you did at the bottom of `weather.py`.
3. The `claude_desktop_config.json` has the weather server from the first workshop. Add another dictionary to include `invoice.py`. 
4. Run the Temporal Server: `temporal server start-dev`.
5. Make sure that you have Claude Desktop downloaded.
6. Copy this configuration to your Claude Desktop config file: `cp claude_desktop_config.json ~/Library/Application\ Support/Claude/`.
7. Restart Claude Desktop.
8. Look for the plugin icon in the chat interface - it should show your invoice server is connected.
9. In Claude Desktop, try prompting it with: `Process this invoice for me: {
  "invoice_id": "INV-100",
  "customer": "ACME Corp",
  "lines": [
    {"description": "Widget A", "amount": 100, "due_date": "2024-06-30T00:00:00Z"},
    {"description": "Widget B", "amount": 200, "due_date": "2024-07-05T00:00:00Z"}
  ]
}`.
10. Allow it to make a call to the `process_invoice` tool.
11. Open `http://localhost:8233` to see your Workflows executing in real-time! Notice the timer has started to represent a long-running Workflow Execution and wait for a Signal.
12. Ask Claude what the status is of the invoice.
13. Allow prompt to use the `invoice_status` tool and point out the status. What Event does this add in your Event History?
14. Finally, prompt Claude to "approve this invoice." What Event does this add in your Event History?
15. Allow Claude to use the `approve_invoice` tool.
16. Go to the Web UI and see that your Workflow Execution has completed successfully.
17. Can you still query the Workflow even though it's completed successfully?

In [36]:
# Kill any worker to prepare for the exercise.
x = worker.cancel()

## Summary: 

- **Durable Timers** - Unlike regular `asyncio.sleep()`, Temporal's timers survive process restarts and continue counting down in the server
- **Human-in-the-Loop Patterns** - Signals allow external systems (including humans) to modify workflow execution, while Queries provide read-only access to current state
- **Parent-Child Workflows** - Complex processes can be broken into smaller, manageable workflows that run in parallel while maintaining durability
- **Durability** - With automatic retries, error handling, and state persistence, Temporal adds durability to long-running MCP tools

## Exercise 2 - Dealing with Long-Running Workflows and HITL

* In this exercise, you will:
  * Implement a Temporal Signal to send external input to your running Workflow Execution
  * Build a Temporal Query to fetch the latest conversion amount from yourWworkflow state
  * Pause Workflow progress so it waits on Signals or timeouts
  * Add durability to long-running workflows by exposing them through MCP tools
* Go to the **exercises** Directory in the Google Drive and open the `02_Long_Running_Workflows_HITL` folder. Then, open the **practice** directory.
* Open the `README.md` and follow the instructions
* If you get stuck, raise your hand and someone will come by and help. You can also check the `Solution` directory for the answers
* **You have 15 mins**